In [ ]:
import pandas as pd
import numpy as np

from numpy.random import multivariate_normal as Gaussian

import time
from scipy.stats import logistic, truncnorm

#from scipy.optimize import minimize

In [ ]:
def sigma(y):
    O = np.zeros(y.shape)
    I = (y>=0)
    J = (y<0)
    O[I] = 1/(1+np.exp(-y[I]))
    O[J] = np.exp(y[J])/(1+np.exp(y[J]))
    return O

In [ ]:
sigma__ = lambda z: logistic.cdf(z)

In [ ]:
data = np.genfromtxt("data/bank-note/train.csv", dtype = float, delimiter = ',')
data.shape

In [ ]:
data = np.hstack((np.ones((data.shape[0],1)), data))
data.shape

In [ ]:
data_train = data[:,:-1]
label_train = data[:,-1].astype(int)
data_train.shape, label_train.shape

In [ ]:
test_ = np.genfromtxt("data/bank-note/test.csv", dtype = float, delimiter = ',')
test_.shape

In [ ]:
test_ = np.hstack((np.ones((test_.shape[0],1)), test_))
test_.shape

In [ ]:
test = test_[:,:-1]
label = test_[:,-1].astype(int)
test.shape, label.shape

In [ ]:
def nll_(x, t, w):
    s_ = x @ w
    y_ = sigma(s_)
    I = (t == 0)
    y_[I] = 1-y_[I]
    y_ = np.log(y_+1e-10)
    return -y_.sum() + 0.5* np.dot(w,w)

In [ ]:
nll = lambda w: nll_(data_train, label_train, w)

In [ ]:
d = data_train.shape[1]
mu = np.zeros(d)
M = np.eye(d)
M_inv = np.linalg.inv(M)

In [ ]:
K = lambda r : 0.5 * r @ M_inv @ r

In [ ]:
U = lambda w : nll(w)

In [ ]:
def dU_(x, t, w):
    y = t - sigma(x@w)
    return w - x.T @ y

In [ ]:
dU = lambda w : dU_(data_train, label_train, w)

In [ ]:
H = lambda w, r : U(w) + K(r)

In [ ]:
def Leapfrog(eps, w, r, dU = dU):
    r_half = r - eps * dU(w)/2
    w_new = w + eps * M_inv @ r_half
    r_new = r_half - eps * dU(w_new)/2
    return w_new, r_new

In [ ]:
def L_step_Leapfrog(w, r, eps, L):
    r_ = r.copy()
    w_ = w.copy()
    for l in range(L):
        w_, r_ = Leapfrog(eps = eps, w = w_, r = r_)
    return w_ , r_

In [ ]:
def HMC(w_0, r_pdf = Gaussian, L = 10, eps = 0.01, 
        n_sample = 10000, burn_in_after = 10**5, pick_every = 10, display = True):
    
    Sample_points = []
    iteration = -1
    c = -1
    sampled = 0
    w_new = w_0.copy()
    
    while sampled <= n_sample or iteration >200000:
        iteration += 1
        r_0 = r_pdf(mu, M)
        w_new, r_new = L_step_Leapfrog(eps = eps, w = w_0, r = r_0, L = L)
        
        log_p = np.log(np.random.uniform())
        log_q = H(w_0, r_0) - H(w_new, -r_new)

        if log_p <= log_q:
            c += 1
            if display and (iteration % 10000 == 0):
                print('Iteration is {}, number of accepted points is {}'.format(iteration, c))
                
            if iteration == burn_in_after:
                print('burn in stage started!')
                
            if iteration >= burn_in_after:
                if sampled % 10 == 0:
                    Sample_points.append(w_new)
                    w_0 = w_new.copy()
                sampled += 1
            else:
                w_0 = w_new.copy()
                    
    return np.array(Sample_points), c/iteration

In [ ]:
#t = time.time()
#S, p = HMC(w_0 = np.zeros(d))
#print(time.time()-t)

In [ ]:
eps_list = [0.005, 0.1, 0.2, 0.5] #The last value is too BIG
L_list = [10, 20, 50]

In [ ]:
def predict(x, t, w):
    y = sigma(x@w)
    l_pred = (y>=0.5).astype(int)
    return (t==l_pred).mean()

In [ ]:
#av = 0
#for i in range(1000):
#    av += predict(x = data_train,t = label_train,w = S[i])
#print('Train predictive accuracy: {}'.format(av/1000))

In [ ]:
#av = 0
#for i in range(1000):
#    av += predict(x = test, t = label, w = S[i])
#print('Test predictive accuracy: {}'.format(av/1000))

In [ ]:
def predictive_likelihood(x, t, w):
    y = sigma(x@w)
    I = (t == 0)
    y[I] = 1 - y[I]
    return y.mean() 

In [ ]:
#av = 0
#for i in range(1000):
#    av += predictive_likelihood(x = test, t = label, w = S[i])
#print('Test predictive likelihood: {}'.format(av/1000))

In [ ]:
S_list = []
for eps in eps_list:
    for L in L_list:
        print('eps: {} and L: {}'.format(eps, L))
        t = time.time()
        S, p = HMC(w_0 = np.zeros(d), L = L, eps = eps)
        print('Running time for eps = {} and L = {} is {}'.format(eps, L, time.time()-t))
        print('Acceptance rate: {}'.format(p))
        S_list.append(S)
        
        av = 0
        mean = np.zeros(d)
        for i in range(1000):
            mean += S[i]/1000
            av += predictive_likelihood(x = test, t = label, w = S[i])
        print('Test predictive likelihood for eps = {} and L = {} is {}'.format(eps, L, av/1000))
            
        av = 0
        for i in range(1000):
            av += predict(x = test, t = label, w = S[i])
        print('Test predictive accuracy for eps = {} and L = {} is {}'.format(eps, L, av/1000))
        print('Emperical mean is: ', mean)

## (b) Gibbs sampling for the Bayesian probit model with augmented variables!

In [ ]:
truncnorm.rvs(a = 0, b= np.inf, loc=0, scale=1, size=1)

In [ ]:
N, d = data_train.shape
S_inv = np.eye(d) + data_train.T @ data_train
S = np.linalg.inv(S_inv)

In [ ]:
def _Z_update(w, x, t):
    Z = np.zeros(N)
    mu = x @ w
    for i in range(N):
            Z[i] = (2*t[i]-1)*truncnorm.rvs(a = 0, b = np.inf, loc=mu[i], scale=1, size=1)
    return Z

In [ ]:
def _w_update(z):
    mu = S @ data_train.T @ z
    w = Gaussian(mean = mu, cov = S)
    return w

In [ ]:
def Gibbs_sampling(x = data_train, t = label_train, burn_in_at = 10**5, n_samples = 1000, pick_every  =10):
    w_ = np.random.normal(0,1, size = d)
    itr = 0
    while itr< burn_in_at:
        z_ = _Z_update(w_, x, t)
        w_ = _w_update(z_)
        itr += 1
    itr = 0
    w_Samples = []
    z_Samples = []
    print('burned in')
    while itr < n_samples * pick_every:
        if itr % pick_every == 0:
            w_Samples.append(w_.copy())
            z_Samples.append(z_.copy())
        z_ = _Z_update(w_, x, t)
        w_ = _w_update(z_)
        itr += 1
    return w_Samples, z_Samples

In [ ]:
W, Z = Gibbs_sampling()

In [ ]:
Z_ = np.array(Z).T
Z_.shape

In [ ]:
P = (Z_>= 0).astype(int)

In [ ]:
ACC = []
for i in range(P.shape[1]):
    pp = ((P[:,i] - label_train) == 0).mean()
    ACC.append(pp)

In [ ]:
np.array(ACC).mean()